In [2]:
%%time
import findspark 
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

spark = SparkSession.builder.appName('i_s3_postgres').getOrCreate()

CPU times: user 316 ms, sys: 229 ms, total: 545 ms
Wall time: 13.8 s


In [2]:
%%time
# from pyspark.sql.types import StructType, StructField
# from pyspark.sql.types import DoubleType, IntegerType, StringType

bucket_name = "aatempjson2"
marca = 'elcomercio'
date_format = '20190301'
campaings = ['mundialista', 'semanasanta']

for campain in campaings:
    path_json_spark = 's3n://{}/{}/{}/{}/*.json'.format(bucket_name, marca, date_format, campain)
    print(path_json_spark)
    df_csv = spark.read.json(path_json_spark)
    df_csv.show()
    break

s3n://aatempjson2/elcomercio/20190301/mundialista/*.json
+-----------+--------+--------+-------------------+
|    campain|    date|   event|                url|
+-----------+--------+--------+-------------------+
|mundialista|20190301|received|http://quipuai.com/|
|mundialista|20190301|received|http://quipuai.com/|
|mundialista|20190301|received|http://quipuai.com/|
|mundialista|20190301|received|http://quipuai.com/|
|mundialista|20190301|received|http://quipuai.com/|
|mundialista|20190301|received|http://quipuai.com/|
|mundialista|20190301|received|http://quipuai.com/|
|mundialista|20190301|received|http://quipuai.com/|
|mundialista|20190301|received|http://quipuai.com/|
|mundialista|20190301|received|http://quipuai.com/|
|mundialista|20190301|received|http://quipuai.com/|
|mundialista|20190301|received|http://quipuai.com/|
|mundialista|20190301|received|http://quipuai.com/|
|mundialista|20190301|received|http://quipuai.com/|
|mundialista|20190301|received|http://quipuai.com/|
|mundia

In [3]:
%%time
df_csv.count()

CPU times: user 32.9 ms, sys: 5.07 ms, total: 38 ms
Wall time: 2min 28s


1000

In [4]:
%%time
events = [_.event for _ in df_csv.select('event').distinct().collect()]

CPU times: user 31.9 ms, sys: 10.7 ms, total: 42.6 ms
Wall time: 2min 20s


In [23]:
%%time
from pyspark.sql.functions import col, asc

dicc_save = {}
for event in events:
    counter = df_csv.where(col("event") == event).count()
    #counter = df_csv.sql(s"SELECT col1 from table where col2>500 limit $q25)
    dicc_save[event] = counter
    print(event, counter)

received 500
ctr 500
CPU times: user 43.4 ms, sys: 11.2 ms, total: 54.7 ms
Wall time: 4min 27s


In [37]:
from pyspark.sql import Row
from collections import OrderedDict

def convert_to_row(d):
    return Row(
        **OrderedDict(sorted(d.items()))
    )

data = sc.parallelize(
    [dicc_save]
).map(convert_to_row).toDF()
data.show()

+---+--------+
|ctr|received|
+---+--------+
|500|     500|
+---+--------+



In [36]:

bucket_output = 'aaoutputattempjson2'
hoy = str(datetime.now()).replace(' ', '-').replace(':', '-').split('.')[0]
path = '{}.csv'.format(hoy)
path

'2019-04-05-14-29-51.csv'

In [40]:
data.write.mode('append').csv('s3n://{}/{}'.format(bucket_output, hoy))

In [ ]:
ddff_validate = spark.read.csv('s3n://notification.output.test/**/*.csv')

ddff_validate.show()